# Deferred Revenue  

### Getting the data

In [1]:
# Libraries needed
import pandas as pd
import numpy as np
import math
import sqlalchemy
from sqlalchemy import create_engine

import json
from datetime import datetime, date, timedelta

# Libraries for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import creds_panoply

# Establishing the connection
POSTGRES_ADDRESS = creds_panoply.address
POSTGRES_PORT = creds_panoply.port
POSTGRES_USERNAME = creds_panoply.username
POSTGRES_PASSWORD = creds_panoply.pw
POSTGRES_DBNAME = creds_panoply.dbname

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
    username=POSTGRES_USERNAME,
    password=POSTGRES_PASSWORD,
    ipaddress=POSTGRES_ADDRESS,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

# Importing selected fields
tbl_name = "cirkul_database_import.business_intelligence_orders"
sel_fields = "shopify_order_id, order_processed_at_date, watched_order_batched_at_date, order_total_sip_count, order_revenue, order_accounting_revenue"
query = "SELECT " + sel_fields + " FROM " + tbl_name
df_orders = pd.read_sql_query(query, cnx)

print("# of Total orders: ", len(df_orders))
print("# of orders with missing batch date: ", df_orders['watched_order_batched_at_date'].isna().sum())
print("Sample (5 records)")
df_orders.head()


# of Total orders:  3272799
# of orders with missing batch date:  1295952
Sample (5 records)


,shopify_order_id,order_processed_at_date,watched_order_batched_at_date,order_total_sip_count,order_revenue,order_accounting_revenue
0,3679989203034,2021-03-12 14:09:37,NaT,17,87.50,87.50
1,3009411481690,2021-01-01 07:12:11,NaT,42,141.81,145.35
2,1362037243994,2019-01-24 00:50:11,NaT,2,28.00,28.00
3,3693081493594,2021-03-17 22:47:26,NaT,0,5.00,10.00
4,1382759694426,2018-05-18 12:54:48,NaT,2,5.00,5.00


### Field Modifications

In [2]:
# Adding Order & Batch month
df_orders['Order_Month'] = df_orders['order_processed_at_date'].dt.strftime('%Y-%m')
df_orders['Batch_Month'] = df_orders['watched_order_batched_at_date'].dt.strftime('%Y-%m')

# Adding columns to identify Missing Batch Dates & if the Batch Month does not equal to the order month
Is_Batch_Date_Missing = []
Is_Fulfilled =[]
Fulfilled_Sip_Count = []
Deferred_Revenue = []

for index, row in df_orders.iterrows():
    if row['watched_order_batched_at_date'] != row['watched_order_batched_at_date']:
        Is_Batch_Date_Missing.append(1)
    else:
        Is_Batch_Date_Missing.append(0)
    
    if row['Order_Month'] == row['Batch_Month']:
        Is_Fulfilled.append(1)
        Deferred_Revenue.append(0)
        Fulfilled_Sip_Count.append(row['order_total_sip_count'])
    else:
        Is_Fulfilled.append(0)
        Deferred_Revenue.append(row['order_accounting_revenue'])
        Fulfilled_Sip_Count.append(0)

# Adding those lists to dataframe
df_orders['Is_Batch_Date_Missing'] = Is_Batch_Date_Missing
df_orders['Is_Fulfilled'] = Is_Fulfilled
df_orders['Deferred_Revenue'] = Deferred_Revenue
df_orders['Fulfilled_Sip_Count'] = Fulfilled_Sip_Count

df_orders.head()

,shopify_order_id,order_processed_at_date,watched_order_batched_at_date,order_total_sip_count,order_revenue,order_accounting_revenue,Order_Month,Batch_Month,Is_Batch_Date_Missing,Is_Fulfilled,Deferred_Revenue,Fulfilled_Sip_Count
0,3679989203034,2021-03-12 14:09:37,NaT,17,87.50,87.50,2021-03,NaN,1,0,87.50,0
1,3009411481690,2021-01-01 07:12:11,NaT,42,141.81,145.35,2021-01,NaN,1,0,145.35,0
2,1362037243994,2019-01-24 00:50:11,NaT,2,28.00,28.00,2019-01,NaN,1,0,28.00,0
3,3693081493594,2021-03-17 22:47:26,NaT,0,5.00,10.00,2021-03,NaN,1,0,10.00,0
4,1382759694426,2018-05-18 12:54:48,NaT,2,5.00,5.00,2018-05,NaN,1,0,5.00,0


### Monthly summary of orders with missing batch dates

In [9]:
orders_Total = pd.DataFrame(df_orders.groupby('Order_Month').count()['shopify_order_id'])
orders_Fulfilled = pd.DataFrame(df_orders.groupby('Order_Month').sum()['Is_Fulfilled'])
orders_Unfulfilled = pd.DataFrame(df_orders.groupby('Order_Month').sum()['Is_Batch_Date_Missing'])
Total_Revenue = pd.DataFrame(df_orders.groupby('Order_Month').sum()['order_accounting_revenue'])
Deferred_Revenue = pd.DataFrame(df_orders.groupby('Order_Month').sum()['Deferred_Revenue'])
Total_Sip_Count = pd.DataFrame(df_orders.groupby('Order_Month').sum()['order_total_sip_count'])
Fulfilled_Sip_Count = pd.DataFrame(df_orders.groupby('Order_Month').sum()['Fulfilled_Sip_Count'])

# Joining these dataframes & renaming the columns
monthly_summary = pd.concat([orders_Total, orders_Fulfilled, orders_Unfulfilled, Total_Revenue, Deferred_Revenue, Total_Sip_Count, Fulfilled_Sip_Count], axis=1)
monthly_summary.rename(columns={'shopify_order_id':'Total Orders (Shopify)',
                                'Is_Fulfilled':'Fulfilled Orders',
                                'Is_Batch_Date_Missing':'Unfulfilled Orders',
                                'order_accounting_revenue':'Total Revenue (Acc.)',
                                'Deferred_Revenue':'Deferred_Revenue (Acc.)',
                                'order_total_sip_count':'Total Sip Count',
                                'Fulfilled_Sip_Count':'Fulfilled Sip Count'}, inplace=True)

# Adding a column for % of unfulfilled orders & % deferred revenue
monthly_summary['% of Unfulfilled Orders'] = (monthly_summary['Unfulfilled Orders']/monthly_summary['Total Orders (Shopify)'])
monthly_summary['% of Deferred Revenue'] = (monthly_summary['Deferred_Revenue (Acc.)']/monthly_summary['Total Revenue (Acc.)'])

# formatting columns
# monthly_summary['Total Orders (Shopify)'] = monthly_summary['Total Orders (Shopify)'].map('{:,}'.format)
# monthly_summary['Fulfilled Orders'] = monthly_summary['Fulfilled Orders'].map('{:,}'.format)
# monthly_summary['Unfulfilled Orders'] = monthly_summary['Unfulfilled Orders'].map('{:,}'.format)
# monthly_summary['Total Revenue (Acc.)'] = monthly_summary['Total Revenue (Acc.)'].map('${:,.0f}'.format)
# monthly_summary['Deferred_Revenue (Acc.)'] = monthly_summary['Deferred_Revenue (Acc.)'].map('${:,.0f}'.format)
# monthly_summary['% of Unfulfilled Orders'] = monthly_summary['% of Unfulfilled Orders'].map('{:,.2%}'.format)
# monthly_summary['% of Deferred Revenue'] = monthly_summary['% of Deferred Revenue'].map('{:,.2%}'.format)
# monthly_summary['Total Sip Count'] = monthly_summary['Total Sip Count'].map('{:,}'.format)
# monthly_summary['Fulfilled Sip Count'] = monthly_summary['Fulfilled Sip Count'].map('{:,}'.format)

# Reordering columns
monthly_summary = monthly_summary.reindex(columns=['Total Orders (Shopify)', 'Fulfilled Orders', 'Unfulfilled Orders', '% of Unfulfilled Orders',
                                 'Total Revenue (Acc.)', 'Deferred_Revenue (Acc.)', '% of Deferred Revenue', 'Total Sip Count', 'Fulfilled Sip Count'])
monthly_summary = monthly_summary.transpose()
monthly_summary

Order_Month,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,...,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05
Total Orders (Shopify),65.0,399.0,693.0,1748.0,6691.0,13953.0,27800.0,23808.0,22714.0,23023.0,...,1.187510e+05,9.964200e+04,9.082500e+04,8.466000e+04,7.870500e+04,1.107790e+05,1.534740e+05,3.313820e+05,2.231300e+05,28089.000000
Fulfilled Orders,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.131230e+05,9.707700e+04,8.681400e+04,8.193800e+04,7.602300e+04,9.548700e+04,1.380750e+05,2.183910e+05,1.205070e+05,1514.000000
Unfulfilled Orders,65.0,399.0,693.0,1748.0,6691.0,13953.0,27800.0,23808.0,22714.0,23023.0,...,1.655000e+03,1.048000e+03,9.750000e+02,1.288000e+03,1.113000e+03,1.169000e+03,3.900000e+02,3.512000e+03,8.101200e+04,26575.000000
% of Unfulfilled Orders,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.393672e-02,1.051765e-02,1.073493e-02,1.521380e-02,1.414141e-02,1.055254e-02,2.541147e-03,1.059804e-02,3.630709e-01,0.946100
Total Revenue (Acc.),457.0,3150.0,12948.6,28184.6,86127.0,202570.2,380781.8,400597.6,423052.5,423346.6,...,3.296215e+06,2.982295e+06,2.742251e+06,2.537862e+06,2.447723e+06,3.529883e+06,4.926884e+06,1.100389e+07,7.562489e+06,949227.890000
Deferred_Revenue (Acc.),457.0,3150.0,12948.6,28184.6,86127.0,202570.2,380781.8,400597.6,423052.5,423346.6,...,1.549305e+05,7.543708e+04,1.113790e+05,8.075438e+04,7.465723e+04,4.727993e+05,5.158422e+05,4.059436e+06,4.118240e+06,917761.490000
% of Deferred Revenue,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,4.700253e-02,2.529497e-02,4.061589e-02,3.181984e-02,3.050068e-02,1.339419e-01,1.046995e-01,3.689092e-01,5.445615e-01,0.966851
Total Sip Count,152.0,974.0,3826.0,8364.0,27136.0,62366.0,119764.0,123618.0,129062.0,130258.0,...,8.813070e+05,8.042700e+05,7.401280e+05,6.536500e+05,6.499210e+05,8.913370e+05,1.202511e+06,2.481621e+06,1.791492e+06,234610.000000
Fulfilled Sip Count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.452020e+05,7.898830e+05,7.128630e+05,6.419740e+05,6.330740e+05,7.796160e+05,1.079131e+06,1.638595e+06,9.646930e+05,9022.000000


### Exporting momthly summary to excel file

In [10]:
# Data export to excel

import platform
import os
os_name = platform.system()

# TimeStamp
dateTimeObj = datetime.now(tz=None)
timeStamp = str(str(dateTimeObj.month) + "." + str(dateTimeObj.day) + "." + str(dateTimeObj.year))

filename = "Unfulfilled Orders & Deferred Revenue by Month_" + timeStamp + ".xlsx"

if os_name == "Darwin":
    desktop_path = os.path.join(os.path.join(os.path.expanduser('~')), 'Downloads')
    out_path = desktop_path + "/" + filename
    with pd.ExcelWriter(out_path) as writer:
        monthly_summary.to_excel(writer, sheet_name='Summary')

if os_name == "Windows":
    desktop_path = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Downloads')
    out_path = desktop_path + "//" + filename
    with pd.ExcelWriter(out_path) as writer:
        monthly_summary.to_excel(writer, sheet_name='Summary')
